# Evoluzione Vaccinazioni Covid19 Italia

In [1]:
import pandas as pd
import os
import git
import sys
sys.path.append('../')
from src.data_downloader import download_from_repo, rmtree
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import ipywidgets as widgets
from ipywidgets import interact, interactive, interactive_output

In [2]:
repopath = 'https://github.com/italia/covid19-opendata-vaccini'
csvlist = ['vaccini-summary-latest.csv', 'somministrazioni-vaccini-summary-latest.csv','somministrazioni-vaccini-latest.csv','consegne-vaccini-latest.csv']
filenames = ['dati/'+file for file in csvlist]
dest = '../data/vaccini'

In [3]:
download_from_repo(repopath, filenames, dest)

updated  dati/vaccini-summary-latest.csv
updated  dati/somministrazioni-vaccini-summary-latest.csv
updated  dati/somministrazioni-vaccini-latest.csv
updated  dati/consegne-vaccini-latest.csv
last commit  2021-04-10 10:09:00


In [4]:
data = {}
datanames = []
for file in os.listdir(dest):
    datanames.append(file.replace('.csv',''))
    data[file.replace('.csv','')] = pd.read_csv(os.path.join(dest,file))

In [5]:
pop = pd.read_csv("../data/popitaregions.csv")
pop_dict = {reg: num for (reg, num) in zip(pop['Regione'], pop['Popolazione'])}
pop_dict['Provincia Autonoma Bolzano / Bozen'] = pop_dict['Trentino-Alto Adige']/2
pop_dict['Provincia Autonoma Trento'] = pop_dict['Trentino-Alto Adige']/2
pop_dict["Valle d'Aosta / Vallée d'Aoste"] = pop_dict["Valle d'Aosta"]
pop_dict["Friuli-Venezia Giulia"] = pop_dict['Friuli Venezia Giulia']

## Quadro di sintesi

In [6]:
df_summary = data['vaccini-summary-latest'].copy()
df_summary['population'] = df_summary['nome_area'].map(pop_dict)
df_summary['dosi_somministrate_per_mln_abitante'] = df_summary['dosi_somministrate']/df_summary['population']
df_summary['dosi_consegnate_per_mln_abitante'] = df_summary['dosi_consegnate']/df_summary['population']
df_summary.loc[-1] = df_summary.sum()
df_summary.loc[-1,"area"] = 'ITA'
df_summary.loc[-1,"nome_area"] = 'Italia'
@interact
def plot_summary(groups=['dosi_assolute', 'dosi_per_abitante'],sort_by=['somministrate','consegnate','percentuale_somministrazione']):    
    if groups == 'dosi_assolute':
        ys = ""
    elif groups == 'dosi_per_abitante':
        ys = "_per_mln_abitante"
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    if sort_by == 'percentuale_somministrazione':
        sortby = 'percentuale_somministrazione'
    else:
        sortby = 'dosi_'+sort_by+ys
    tempdf= df_summary.sort_values(sortby, ascending=False)
    fig.add_trace(go.Bar(x=tempdf['nome_area'],y=tempdf['dosi_somministrate'+ys], name='dosi somministrate'),secondary_y=False)
    fig.add_trace(go.Bar(x=tempdf['nome_area'],y=tempdf['dosi_consegnate'+ys], name='dosi consegnate'),secondary_y=False)
    fig.update_yaxes(title_text="<b>dosi</b>", secondary_y=False)
    fig.update_yaxes(title_text="<i>percentuale</i>", secondary_y=True)
    fig.add_trace(go.Scatter(x=tempdf['nome_area'],y=tempdf['percentuale_somministrazione'],name='perc somministrata'),secondary_y=True)  
    fig.show()


interactive(children=(Dropdown(description='groups', options=('dosi_assolute', 'dosi_per_abitante'), value='do…

## Andamento Distribuzione Vaccini per regioni

In [7]:
df_consegne = data['consegne-vaccini-latest'].groupby('nome_area')
@interact
def plot_forniture(region = list(df_consegne.groups.keys())):    
    plotdf = df_consegne.get_group(region)[['data_consegna','fornitore','numero_dosi']]
    fig = px.bar(plotdf, x='data_consegna', y="numero_dosi", color="fornitore", title="Forniture Vaccini per "+region)
    fig.show()

interactive(children=(Dropdown(description='region', options=('Abruzzo', 'Basilicata', 'Calabria', 'Campania',…

## Andamento Somministrazioni per regione

In [8]:
df_somm = data['somministrazioni-vaccini-latest'].groupby('nome_area')

@interact
def plot_somm(region = ['Italia']+list(df_somm.groups.keys()),groups=['sex', 'categories', 'stages']):
    if region=='Italia':
        plotdf = data['somministrazioni-vaccini-summary-latest'].groupby('data_somministrazione').sum().drop('codice_regione_ISTAT',1)
    else:
        plotdf = df_somm.get_group(region).groupby('data_somministrazione').sum().drop('codice_regione_ISTAT',1)

    if groups == 'sex':
        ys = [col for col in plotdf.columns if 'sesso' in col]
    elif groups == 'categories':
        ys = [col for col in plotdf.columns if 'categoria' in col]
    elif groups == 'stages':
        ys = [col for col in plotdf.columns if 'dose' in col]
    fig = px.bar(plotdf, x=plotdf.index, y=ys, title="somministrazioni per "+region)
    fig.show()

interactive(children=(Dropdown(description='region', options=('Italia', 'Abruzzo', 'Basilicata', 'Calabria', '…